In [1]:
#!/usr/bin/env python
# coding: utf-8
import sys
import scanpy as sc
import anndata
import pandas as pd
import numpy as np
import os
import gc
import decoupler as dc
import regex as re
import math
os.chdir("/Users/ricardoramirez/Dropbox/PhD/Research/MOFAcell")

In [3]:
sc_dat = sc.read_h5ad(filename = "/Volumes/RicoData2/MI_project/MI_revisions/HCA_submission/snrnaseq/hca/hca_rnasamples.h5ad")

In [4]:
sc_dat = sc_dat[~sc_dat.obs["cell_state"].isna().values]

In [5]:
sc_dat.layers['counts'] = sc_dat.X

In [6]:
cell_type_numbers = sc_dat.obs.groupby(["patient_region_id", "cell_state"])["cell_state"].count()
cell_type_numbers = cell_type_numbers.to_frame().rename({"cell_state":"counts"}, axis=1).reset_index()

In [7]:
padata = dc.get_pseudobulk(sc_dat, sample_col='patient_region_id', groups_col='cell_state', layer='counts', min_prop=0, min_smpls=0)

/Users/ricardoramirez/opt/miniconda3/envs/sc_py/lib/python3.9/site-packages/decoupler/utils_anndata.py:203: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  psbulk = AnnData(psbulk, obs=new_obs, var=new_var)


In [8]:
pb_dat = pd.DataFrame(padata.X)
pb_dat.columns = padata.var.index.values
pb_dat.index = padata.obs.index.values
pb_dat.to_csv("./data_MI/pb_data_states.csv")

In [9]:
pb_coldata = padata.obs.copy()
pb_coldata["colname"] = pb_coldata.index.values
pb_coldata = pb_coldata.merge(cell_type_numbers, on = ["cell_state","patient_region_id"], how = "left")

In [11]:
pb_coldata.to_csv("./data_MI/pb_coldata_states.csv")

In [10]:
pb_coldata

doublet cell_type     cell_state patient_region_id patient patient_group  \
0    singlet      Endo  Arterial_Endo         FZ/GT_P19     P19       group_3   
1    singlet      Endo  Arterial_Endo          FZ/GT_P4      P4       group_3   
2    singlet      Endo  Arterial_Endo            FZ_P14     P14       group_3   
3    singlet      Endo  Arterial_Endo            FZ_P18     P18       group_3   
4    singlet      Endo  Arterial_Endo            FZ_P20     P20       group_3   
..       ...       ...            ...               ...     ...           ...   
603  singlet     vSMCs        vSMCs_2             RZ_P9      P9       group_1   
604  singlet     vSMCs        vSMCs_2        control_P1      P1       group_1   
605  singlet     vSMCs        vSMCs_2       control_P17     P17       group_1   
606  singlet     vSMCs        vSMCs_2        control_P7      P7       group_1   
607  singlet     vSMCs        vSMCs_2        control_P8      P8       group_1   

    major_labl                  colname  counts  
0           FZ  FZ/GT_P19_Arterial_Endo      96  
1           FZ   FZ/GT_P4_Arterial_Endo     226  
2           FZ     FZ_P14_Arterial_Endo     253  
3           FZ     FZ_P18_Arterial_Endo     151  
4           FZ     FZ_P20_Arterial_Endo      94  
..         ...                      ...     ...  
603         RZ            RZ_P9_vSMCs_2      30  
604       CTRL       control_P1_vSMCs_2      43  
605       CTRL      control_P17_vSMCs_2      40  
606       CTRL       control_P7_vSMCs_2      25  
607       CTRL       control_P8_vSMCs_2      50  

[608 rows x 9 columns]